In [1]:
''' Do not change this cell '''
#Functions

def check_gpu():
  if tf.test.gpu_device_name() != '':
    print("GPU sucessfully connected")
  else: print("PLease connect GPU")



# **#Task 1**#


In [2]:
#import all the required packages and  libraries for FineTuning LLM
!pip install transformers datasets accelerate bitsandbytes
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import tensorflow as tf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
''' Do not change this cell '''
check_gpu()


GPU sucessfully connected


#**Task #2 notebook**#

In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `ColabToken` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `ColabTo

In [ ]:
# !pip install -q datasets
# from datasets import load_dataset

splits = {'train': 'train.csv', 'validation': 'validation.csv', 'test': 'test.csv'}
df = load_dataset('jizzu/llama2_indian_law_v2')
dfTrain = df['train']
dfVal = df['validation']
dfTest = df['test']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train.csv:   0%|          | 0.00/13.4M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/900k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/112k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/24607 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/455 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/276 [00:00<?, ? examples/s]

In [ ]:
print(dfTrain[0])
print(type(dfTrain[0]['text']))

{'text': "###Human:\nWhat is the difference between a petition and a plaint in Indian law?\n###Assistant:\nA petition is a formal request submitted to a courttribunalor authority to seek a specific remedy or relief. It is commonly used for various purposessuch as filing a writ petition in the High Court or submitting a petition for divorce. On the other handa plaint is a formal written statement of a plaintiff's claim in a civil lawsuit. The key difference is that a petition is more versatile and can be used for various legal matterswhile a plaint is specific to civil cases."}
<class 'str'>


In [ ]:
def preprocessDataset(df,formattedList):

    for rowData in df.iterrows():
        question = rowData[1][0].split("###Human:")[1].split("###Assistant:")[0].strip()
        answer = rowData[1][0].split("###Assistant:")[1].strip()
        formattedList.append({"<s>[INST] {"+question+"} [/INST]{"+answer+"}</s>"});


In [ ]:
def transform_conversation(example):
    conversation_text = example['text']
    segments = conversation_text.split('###')

    reformatted_segments = []

    # Iterate over pairs of segments
    for i in range(1, len(segments) - 1, 2):
        human_text = segments[i].strip().replace('Human:', '').strip()

        # Check if there is a corresponding assistant segment before processing
        if i + 1 < len(segments):
            assistant_text = segments[i+1].strip().replace('Assistant:', '').strip()

            # Apply the new template
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text} </s>')
        else:
            # Handle the case where there is no corresponding assistant segment
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

    return {'text': ' '.join(reformatted_segments)}

In [ ]:
formattedTestList = dfTest.map(transform_conversation)
formattedTrainList = dfTrain.map(transform_conversation)
formattedValList = dfVal.map(transform_conversation)

Map:   0%|          | 0/276 [00:00<?, ? examples/s]

Map:   0%|          | 0/24607 [00:00<?, ? examples/s]

Map:   0%|          | 0/455 [00:00<?, ? examples/s]

In [ ]:
formattedTestList.push_to_hub("llama2_indian_law_v2_formatted_data",split="Test")
formattedTrainList.push_to_hub("llama2_indian_law_v2_formatted_data",split="Train")
formattedValList.push_to_hub("llama2_indian_law_v2_formatted_data",split="Validation")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/25 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/266 [00:00<?, ?B/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/374 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/QuantTitan/llama2_indian_law_v2_formatted_data/commit/f999cc3476e841912811a2d4194a73ed5f0447e6', commit_message='Upload dataset', commit_description='', oid='f999cc3476e841912811a2d4194a73ed5f0447e6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/QuantTitan/llama2_indian_law_v2_formatted_data', endpoint='https://huggingface.co', repo_type='dataset', repo_id='QuantTitan/llama2_indian_law_v2_formatted_data'), pr_revision=None, pr_num=None)

# **#Task 3**
1.Store the path of the preprocessed dataset to a string named dataset_name
<br>
2.Specify the pre-Trained llm from hugging face hub to string named model_name
<br>
3.Define the required parameters for fine tuning the llm ((these definitions will be used further)).
<br>
refer this resource : 1.https://huggingface.co/docs/peft/v0.13.0/en/package_reference/lora#peft.LoraConfig
<br>
2.https://arxiv.org/abs/2106.09685





In [5]:
#code goes here
dataset_name = "QuantTitan/llama2_indian_law_v2_formatted_data"
model_name = "meta-llama/Llama-2-7b-chat-hf"

In [6]:
# from transformers import AutoModelForSeq2SeqLM
!pip install peft trl
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig


# peft_config = LoraConfig(
#     lora_alpha=32,
#     lora_dropout=0.01,
#     r=8,
#     bias="none",
#     task_type="CAUSAL_LM",
# )


# model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
# lora_model = LoraModel(model, config, "default")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 8.5 MB/s eta 0:00:00


In [7]:
loraconfig = LoraConfig(
    task_type="CAUSAL_LM",
    r=4,
    lora_alpha=32,
    # target_modules=["q", "v"],
    lora_dropout=0.01,
)

1. Define parameters for quantization technique, especially use 4 bit quantization method , Select quantization type that will increase the efficiency.(these definitions will be used further)
2. Refer to this resource for guidance : https://huggingface.co/blog/4bit-transformers-bitsandbytes


In [8]:
# from transformers import AutoModelForCausalLM

# new_model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True, device_map="auto")

from transformers import BitsAndBytesConfig


nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

new_model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=nf4_config)
new_model.config.use_cache = False



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

1. Set up the training parameters(no. of epochs , tarin,eval batch size , learning rate etc )
2. refer to this resource for guidance : https://learn.deeplearning.ai/courses/finetuning-large-language-models/lesson/6/training-process
<br>https://arxiv.org/html/2408.13296v1#Ch5.S3 <br>

3.Name the  directory to save model checkpoints as output_dir

In [12]:
output_dir = "./output_dir"

# training_arguments = TrainingArguments(
#     output_dir=output_dir,
#     num_train_epochs=num_train_epochs,
#     train_batch_size=per_device_train_batch_size,
#     gradient_accumulation_steps=gradient_accumulation_steps,
#     optim=optim,
#     save_steps=save_steps,
#     logging_steps=logging_steps,
#     learning_rate=learning_rate,
#     weight_decay=weight_decay,
#     max_grad_norm=max_grad_norm,
#     max_steps=max_steps,
#     warmup_ratio=warmup_ratio,
#     group_by_length=group_by_length,
#     lr_scheduler_type=lr_scheduler_type,
#     report_to="tensorboard"
# )

training_args = TrainingArguments(
    output_dir = output_dir,
    warmup_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=1000,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    # logging_steps=25,
    logging_steps = 50,
    logging_dir="./logs",
    # save_strategy="steps",
    # save_steps=25,
    # evaluation_strategy="steps",
    eval_steps=25,
    do_eval=True,
    gradient_checkpointing=True,
    report_to="none",
    overwrite_output_dir = 'True',
    group_by_length=True,
)


1. Define Supervised Fine Tuning Parameters (SFT) required for fine tuning the llm .
2. refer to this resource for getting the parameters : https://huggingface.co/docs/trl/en/sft_trainer

In [10]:
train_dataset = load_dataset(dataset_name, split="Train")
eval_dataset = load_dataset(dataset_name, split="Validation")

tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.model_max_length = 512

README.md:   0%|          | 0.00/489 [00:00<?, ?B/s]

Test-00000-of-00001.parquet:   0%|          | 0.00/45.1k [00:00<?, ?B/s]

Train-00000-of-00001.parquet:   0%|          | 0.00/4.37M [00:00<?, ?B/s]

Validation-00000-of-00001.parquet:   0%|          | 0.00/223k [00:00<?, ?B/s]

Generating Test split:   0%|          | 0/276 [00:00<?, ? examples/s]

...

Generating Validation split:   0%|          | 0/455 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [13]:
trainer = SFTTrainer(
    model=new_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=loraconfig,
    # dataset_text_field="text",
    # max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_args,
    # packing=packing,
)

trainer.train()

<ipython-input-13-2fb58a0f2d03>:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/24607 [00:00<?, ? examples/s]

Map:   0%|          | 0/455 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
50,1.510400
100,1.168800
150,1.095100
200,1.092100
250,1.094900
300,1.135500
350,1.070200
400,1.086400
450,1.048400
500,1.061500


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=1000, training_loss=1.073092140197754, metrics={'train_runtime': 10878.7685, 'train_samples_per_second': 0.368, 'train_steps_per_second': 0.092, 'total_flos': 1.9842321099546624e+16, 'train_loss': 1.073092140197754, 'epoch': 0.16255537042305035})